## Setup

In [1]:
import sys
sys.path.append('../')
del sys
%reload_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict
import matplotlib.pyplot as plt
from nltk.corpus import stopwords as nltk_stopwords

from database_creation.database import Database

## Parameters

In [3]:
database = Database()

## Load the data

In [4]:
database.process_results()

Processing the results...
Computing the tasks...
Initial length of tasks: 0
Object loaded from ../results/../results/task_answers/v2_0/task/queries_size10k_shuffle_articles1_queries1_seed0.pkl
Object loaded from ../results/../results/task_answers/v2_1/task/queries_sizemax_shuffle_articles1_queries1_seed0.pkl
Object loaded from ../results/../results/task_answers/v2_2/task/queries_sizemax_shuffle_articles1_queries1_seed0.pkl
Final length of tasks: 3
Done (elapsed time: 1s).

Computing the results...
Initial length of results: 0
Object loaded from ../results/task_answers/v2_0/results/batch_00_complete.csv
Object loaded from ../results/task_answers/v2_0/results/batch_01_complete.csv
Object loaded from ../results/task_answers/v2_1/results/batch_03_complete.csv
Object loaded from ../results/task_answers/v2_1/results/batch_02_complete.csv
Object loaded from ../results/task_answers/v2_2/results/batch_04_complete.csv
Object loaded from ../results/task_answers/v2_2/results/batch_05_complete.csv


## Set up

In [5]:
assert database.results.keys() == database.tasks.keys()
tasks, results, versions = dict(), dict(), list(database.results.keys())

for version in versions:
    task, result = database.tasks[version], database.results[version]
    assert not set(results.keys()).intersection(set(result.keys()))
    
    tasks.update(task)
    results.update(result)

print("{} tasks available, {} tasks answered for {} different responses".format(len(tasks), len(results), sum([len(r) for _, r in results.items()])))


61056 tasks available, 2100 tasks answered for 6299 different responses


## Non frequent workers issue

Check the number of answers for each minimal number of answer per worker threshold:

In [6]:
worker_ids = defaultdict(int)

for _, result_list in results.items():
    for result in result_list:
        worker_ids[result.worker_id] += 1

for threshold in range(1, 15):
    worker_ids_truncated = sorted([(count, id_) for id_, count in worker_ids.items() if count <= threshold])
    total_count = sum([count for count, _ in worker_ids_truncated])

    print("Workers that did less than (or exactly) {} examples: {} (for {} answers, {}% of the total)".format(threshold, len(worker_ids_truncated), total_count, round(100*len(worker_ids_truncated)/sum([len(r) for _, r in results.items()]),2)))

Workers that did less than (or exactly) 1 examples: 10 (for 10 answers, 0.16% of the total)
Workers that did less than (or exactly) 2 examples: 16 (for 22 answers, 0.25% of the total)
Workers that did less than (or exactly) 3 examples: 21 (for 37 answers, 0.33% of the total)
Workers that did less than (or exactly) 4 examples: 21 (for 37 answers, 0.33% of the total)
Workers that did less than (or exactly) 5 examples: 25 (for 57 answers, 0.4% of the total)
Workers that did less than (or exactly) 6 examples: 25 (for 57 answers, 0.4% of the total)
Workers that did less than (or exactly) 7 examples: 26 (for 64 answers, 0.41% of the total)
Workers that did less than (or exactly) 8 examples: 27 (for 72 answers, 0.43% of the total)
Workers that did less than (or exactly) 9 examples: 29 (for 90 answers, 0.46% of the total)
Workers that did less than (or exactly) 10 examples: 31 (for 110 answers, 0.49% of the total)
Workers that did less than (or exactly) 11 examples: 32 (for 121 answers, 0.51% 

Chose the threshold:

In [7]:
threshold = 3

Delete the corresponding answers:

In [8]:
to_del_worker_id = [id_ for id_, count in worker_ids.items() if count <= threshold]
to_del_task_id = []

for id_, result_list in results.items():
    for result in result_list:
        if result.worker_id in to_del_worker_id:
            to_del_task_id.append(id_)

print("Tasks to delete: {}, {} different responses".format(len(to_del_task_id), sum([len(r) for _, r in results.items()])))

for id_ in to_del_task_id:
    print("Deleting: "'; '.join([str(r) for r in results[id_] if r.worker_id in to_del_worker_id]))
    results[id_] = [r for r in results[id_] if r.worker_id not in to_del_worker_id]

print("Tasks deleted: {} different responses".format(sum([len(r) for _, r in results.items()])))


Tasks to delete: 37, 6299 different responses
Palestinian groups
[bug]
the Russian politicians
[bug]
diplomat
Politicians
Minister of Defence
[bug]
Islamist
American Internet entrepreneur
THE CHESS CHAMPIONS/THE WHITE AND BLACK
both are politlca entities.
Both comedians from long ago .
affected by the proposed oil pipeline
the two world powers
Evo Morales and Hugo Salvatierra
The two football executives
The two technology companies
the affected countries
The former House members
Both groups have a military wing.
Both countries are in Western Asia.
political entities
These states
dancers/performers
Republican politicians/ex-attorneys
The defendants
The businessmen
The entertainers
artists/artisans
The British politicians
The Eastern Seaboard
The creative men
the professional dancers/the male ballerinas
The writers/theatrical people
The colleagues/The Supreme Court colleagues
the boxers/the pugilist
Tasks deleted: 6262 different responses


## NAs

Number of issues of reported bugs:

In [23]:
cmpt_1, cmpt_2, cmpt_3 = 0, 0, 0

for _, task_results in results.items():
    for result in task_results:
        if result.bug and result.answer1 is not None:
            cmpt_1 += 1
            print("Issue 1 (bug = True with an answer): ", result.answer1)
        
        elif not result.bug and result.answer1 is None:
            cmpt_2 += 1
        
        elif result.bug and result.answer1 is None:
            cmpt_3 += 1

print("{} and {} badly reported issues, against {} correctly reported.".format(cmpt_1, cmpt_2, cmpt_3))

0 and 0 badly reported issues, against 1263 correctly reported.


Correction of the inconsistencies (discard answers when there is one):

In [24]:
for _, task_results in results.items():
    for result in task_results:
        if result.bug and result.answer1 is not None:
            result.answer1 = None
        
        elif not result.bug and result.answer1 is None:
            result.bug = True

for _, task_results in results.items():
    for result in task_results:
        if (result.bug and result.answer1 is not None) or (not result.bug and result.answer1 is None):
            raise Exception

print("No more inconsistencies.")

No more inconsistencies.


Analyze the number of valid answers (ie not bugs) per task:

In [11]:
valid = [0, 0, 0, 0]
valid_ids, bug_ids = [], []

for id_, task_results in results.items():
    cmpt = 0
    
    for result in task_results:
        if not result.bug:
            cmpt += 1
    
    valid[cmpt] += 1
    
    if cmpt >= 2:
        valid_ids.append(id_)
    else:
        bug_ids.append(id_)

print("Number of perfect/good examples: {}/{}".format(valid[3], valid[2]))
print("Number of bad/awful examples: {}/{}".format(valid[1], valid[0]))
print("Number of accepted tasks: {}, number of rejected: {} ({}% accepted)".format(len(valid_ids), len(bug_ids), round(100*len(valid_ids)/(len(valid_ids) + len(bug_ids)))))

Number of perfect/good examples: 1261/464
Number of bad/awful examples: 288/87
Number of accepted tasks: 1725, number of rejected: 375 (82% accepted)


## Observation of the results

Good results

In [12]:
for id_ in valid_ids:
    print(tasks[id_])
    for result in results[id_]:
        print(result)
    print()

Afghanistan and Iran:
[...] Still, Western and Afghan officials say that, beyond its much-trumpeted reconstruction program, Iran is also engaging in a range of activities it is less eager to publicize. Qari Ahmad Ali, a Shiite commander once backed by Iran [Iran], said that since 2001, his former patrons had funneled millions of dollars to a web of Shiite religious schools and charities in western Afghanistan [Afghanistan]. [...]

[bug]
The two Asian countries
The Asian countries

Ethiopia and Somalia:
[...] The situation in Somalia [Somalia] has turned from bad to worse,'' the statement said.'' Ethiopia [Ethiopia] has been patient so far. [...]

the African countries
The two African countries
The two African countries

China and Russia:
[...] The United States circulated a softened draft resolution to the Security Council in response to North Korea 's assertion that it conducted a nuclear test on Monday. The United States pressed for a vote by Friday, but China [China] and Russia [Rus


french politicians
the French politicians/the French presidential candidates
the politicians

Gaza Strip and West Bank:
[...] The man many expect to become the new Palestinian prime minister, Ismail Haniya of the militant Islamic group Hamas, on Friday criticized Israeli proposals to restrict the movement of money, people and goods into and out of the Gaza Strip [Gaza Strip] and the West Bank [West Bank] under a Hamas-run Palestinian Authority.'' These Israeli decisions are part of the policy of repression, terrorism and collective punishment against our people,'' Mr. Haniya said after leaving Friday Prayer in Gaza City.'' [...]

the Mediterranean region
the Mediterranean areas/the places near the Mediterranean
political powers

Larry Silverstein and Michael R. Bloomberg:
[...] Three somewhat exasperated politicians -- New York 's governor, George Pataki; New Jersey 's governor, Jon Corzine; and New York City 's mayor, Michael Bloomberg [Michael R. Bloomberg] -- were speaking with one


The two ex coworkers
the two employees
the journalists/the reporters

Florida and Ten Thousand Islands:
[...] NEAR the end of'' Bone by Bone,'' the final volume of Peter Matthiessen 's trilogy of historical novels tracing the brutal life and violent death of a reluctant serial killer in frontier Florida [Florida], the protagonist drives his boat in circles somewhere near Pavilion Key. He 's not lost in any geographical sense; after decades of living in Ten Thousand Islands [Ten Thousand Islands], Ed Watson knows precisely where he is on the planet. [...]

the southern locations/the American areas
the pieces of land
Southernmost parts of the USA

Anucha Browne Sanders and Isiah Thomas:
[...] Nardiello, operating in a sport where money is hardly the name of the game, told The Associated Press that he is the victim of a political sham. Thomas 's [Isiah Thomas] defense is that Browne Sanders [Anucha Browne Sanders] is shaking him [Anucha Browne Sanders] down.'' [...]

the basketball playe

political powers

Estonia and Russia:
[...] The move, though not yet final, has provoked indignation and even threats of sanctions in Russia [Russia], this country 's ever-looming neighbor, which claims historic dominion, if not political sovereignty, over the fate of the Bronze Soldier. Russia 's [Russia] foreign minister, Sergey V. Lavrov, denounced Estonia 's [Estonia] decision as a blasphemy against the soldiers who defeated Nazi Germany. [...]

The two countries
european countries
the countries

Bill T. Jones, Jack O'Brien, Jennifer Ehle, John Gallagher, Julie White and Tom Stoppard:
[...] At the pre-broadcast ceremony, the Alliance Theater in Atlanta was given the annual regional theater award, the only recognition for work outside Broadway. The 2007 Tony Winners Play:'' The Coast of Utopia,'' Tom Stoppard [Tom Stoppard] Musical:'' Spring Awakening'' Book of a musical:'' Spring Awakening,'' Steven Sater Original score:'' Spring Awakening'' (music by Duncan Sheik; lyrics by Steven

[bug]
the bands
The musicians

John G. Rowland and Lawrence E. Alibozek:
Judge Peter C Dorsey sentences Lawrence E Alibozek [Lawrence E. Alibozek], aide to former Gov John G Rowland [John G. Rowland] [Lawrence E. Alibozek], to five years' probation after he pleaded guilty in federal court 2004 to receiving gifts in exchange for steering state contracts. Prosecutors say Alibozek [Lawrence E. Alibozek] cooperated with investigation of corruption that resulted in resignation and criminal trial of Rowland [John G. Rowland].

The convicted felons...
the two political players
[bug]

Durham, North Carolina and Raleigh:
[...] THE Mayberry-meets-M.I.T. appeal of the Raleigh [Raleigh], Durham [Durham] and Chapel Hill region of North Carolina [North Carolina] blooms in spring and summer, when families can bask in the region 's generous outdoor spaces, its culture and its barbecue. Music and art festivals occupy river-cut parks and lush town squares, the Durham Bulls are running around the basebal

Bad results

In [13]:
for id_ in bug_ids:
    print(tasks[id_])
    for result in results[id_]:
        print(result)
    print()

Arthur Wood and Brent Porter:
[...] While conceding that parts of the building were not fireproofed and lacked the requisite permits, Mr. Porter [Brent Porter] and some of his [Brent Porter] students set out to prove that the house was structurally sound. Using Mr. Wood 's [Arthur Wood] plans and a series of interior photographs taken by his [Arthur Wood] son Christopher -- all currently on view in an exhibition at Pratt -- they documented the building 's condition in detail.'' [...]

[bug]
The two opposing parties
[bug]

Nicholas Minucci and Richard L. Buchter:
[...] In State Supreme Court in Queens yesterday, Justice Buchter [Richard L. Buchter] said Mr. Minucci'' repeatedly used the 'n' word in ways and in a manner that can best be described as an affront to Glenn Moore 's worth as a human being.'' Then he [Richard L. Buchter] sentenced Mr. Minucci [Nicholas Minucci] to 15 years for first-degree robbery as a hate crime, as well as prison terms for additional charges, to be served co

Unique answers for each task

In [14]:
stopwords = set(nltk_stopwords.words('english'))
stopwords.update({'two', 'three', 'four', 'five', 'six'})

tuple_answers = defaultdict(list)

for id_ in valid_ids:
    res = []
    
    for result in results[id_]:
        if not result.bug:
            answer1 = ' '.join([w for w in result.answer1.lower().split() if w not in stopwords])
            answer2 = ' '.join([w for w in result.answer2.lower().split() if w not in stopwords]) if result.answer2 else None
        
            res.append(answer1)
            res.append(answer2) if answer2 else None

    entities = ', '.join(tasks[id_].entities)
    tuple_answers[entities].append(res)
            
    print(entities, ' -> ', ', '.join(set([r+' ['+str(res.count(r))+']' if res.count(r)>1 else r for r in res])))
    
    

Afghanistan, Iran  ->  asian countries [2]
Ethiopia, Somalia  ->  african countries [3]
China, Russia  ->  security council members, east asian countries, asian countries
Joseph I. Lieberman, Ned Lamont  ->  americans, democrats [2], american politicians, politicians
European Union, Warsaw Stock Exchange  ->  economic groups, organizations
Bamako, Mali, New York City  ->  established areas, populations
Chris Brady, Craig Wichman, Wendy Herlich  ->  actors, performers
Myanmar, Thailand  ->  asian countries [2], asian cities
Ismail Haniya, Raymond Johansen  ->  leaders, politician, politicians
Concetta Tomei, Vanessa Aspillaga  ->  entertainers, performers, actresses [2]
Ethiopia, Mogadishu  ->  african places, african regions [2], african areas
Senate, Supreme Court  ->  political entities, federal bodies, parts government
Drew Keith, Gerard J. Arpey  ->  men [2]
Lawrence E. Alibozek, Peter N. Ellef, William A. Tomasso  ->  white collar criminal, acquaintances
Mogadishu, Somalia  ->  as

Adel Yahya, Hussain Osman, Ramzi Mohamed, Yassin Omar  ->  alleged attempted bombers, terrorists, defendants
Christopher L. Jacobs, John J. Faso, William F. Weld  ->  politicians [3]
China, Taiwan  ->  rival nations, asian countries, east asian countries
Jane Jacobs, Robert Moses  ->  new yorkers, americans [2]
Cesar Millan, Mark Derr  ->  dog trainers, dog enthusiasts, dog workers
Iran, Syria  ->  middle eastern countries, western asian countries, countries
Kazue Fukiishi, Sion Sono  ->  contributors film, entertainers [2]
Milwaukee County, Oregon, Rhode Island  ->  american places, american locations, regions, pension cutters
Arthur Pinajian, Larry Joseph, Thomas Schultz  ->  home buyers, renovators
Belarus, Europe, Hungary, Poland, Slovakia, Ukraine  ->  political powers, european areas, european region, oil receiving areas
Geffen Records, Suretone Records  ->  record labels., record labels [2]
Pittsburgh, Washington  ->  political powers, areas united states, u.s. cities, seats gov

Beatrix Potter, Renee Zellweger  ->  american female artists, producers, entertainers
Barry Diller, Frank Gehry  ->  americans, businessmen, american professionals
Czech Republic, Germany, Poland, Russia, Slovakia, Ukraine  ->  european countries, european nations, countries supplied oil pipeline, countries affected shipment disruption
Richard B. Evans, Yves Fortier  ->  businessmen, alcan executives
New York City, Pennsylvania  ->  political powers, eastern states, major united states city state nearby
Citigroup Inc., New York Mets  ->  companies working together, organizations making deal new stadium
Cecilia Sarkozy, Francois Hollande  ->  politicians, adults
Arthur M. Schlesinger Jr., John Fitzgerald Kennedy  ->  americans, american public figures
Alfred Uhry, Donna Murphy, Harold Prince  ->  producers, entertainers, theater participants
Charles E. Schumer, Hillary Rodham Clinton  ->  american politicians [2], lawyers, politicians
Belarus, Russia  ->  european countries, countries, 

Unique answers for each tuple:

In [15]:
sorted_answers = [(len([answer for answer_list in answers for answer in answer_list]),
                   entities,
                   answers)
                  for entities, answers in tuple_answers.items()]
sorted_answers = sorted(sorted_answers, reverse=True)

for count, entities, answers in sorted_answers:
    flattened_answers = [answer for answer_list in answers for answer in answer_list]
    flattened_unique_answers = set(flattened_answers)
    
    print(entities, ' (unique/total answers {}/{})'.format(len(flattened_unique_answers), len(flattened_answers)))
    
    answers_counts = set([(flattened_answers.count(a), a) for a in flattened_answers])
    sorted_answers_counts = sorted(answers_counts, reverse=True)
    
    print(', '.join([answer+' ['+str(count)+']' for count, answer in sorted_answers_counts]), '\n')


Iran, Iraq  (unique/total answers 23/88)
western asian countries [18], middle eastern countries [14], nations [11], neighboring countries [10], countries [10], western asia countries [5], middle eastern powers [2], middle eastern nations [2], countries western asia [2], western-asia countries [1], western asian companies [1], western asia nations [1], western asia countires [1], west asian countries [1], republic iraq [1], political powers [1], neighboring middle eastern countries [1], neighboring hotspots [1], neighboring contries [1], middle eastern republics [1], middle east nations... [1], middle east nations [1],  [1] 

Israel, Lebanon  (unique/total answers 18/72)
countries [18], middle eastern countries [15], western asian countries [13], nations [7], western asia countries [4], middle eastern powers [3], western asian nations [1], western asia neighbors [1], warring nations [1], neighbors syria [1], middle eastern rivals [1], middle eastern nations [1], middle east countries [1

technologists [1], media persons [1], internet moguls [1], entrepreneurs [1] 

Chad Hedrick, Shani Davis  (unique/total answers 3/4)
speed skaters [2], male speed skaters [1], americans [1] 

Carl C. Icahn, Edward S. Lampert, Warren E. Buffett  (unique/total answers 3/4)
businessmen [2], investors [1], american business men. [1] 

Canada, Turkey  (unique/total answers 2/4)
countries [3], nations [1] 

Canada, France, Italy, United States  (unique/total answers 3/4)
countries [2], wealthy g-8 nations [1], nations [1] 

California, Florida  (unique/total answers 4/4)
states [1], large american states [1], coastal states [1], american states [1] 

C. K. Chung, C. Y. Choi, D. S. Kim, K. C. Suh  (unique/total answers 4/4)
hynix employees serving time [1], hynix employees [1], guilty parties [1], convicted hynix managers/directors [1] 

Broad Institute, Starr Foundation, Weill Cornell Medical College  (unique/total answers 4/4)
medical research facilities [1], medical foundations [1], instit

David Armand Caputo, Stephen J. Friedman  (unique/total answers 3/3)
university presidents [1], americans [1], american organizational leaders [1] 

Dave Pelz, Phil Mickelson  (unique/total answers 3/3)
professional golfers [1], golf professionals [1], golf icons [1] 

Darren Lynn Bousman, Shawnee Smith  (unique/total answers 3/3)
show business personalities [1], people involved show business [1], figures entertainment industry [1] 

Daniele Mastrogiacomo, Romano Prodi  (unique/total answers 2/3)
italians [2], news people [1] 

Czech Republic, Russia, Slovakia  (unique/total answers 2/3)
countries [2], european countries [1] 

Czech Republic, Poland, Slovakia  (unique/total answers 3/3)
european nations [1], countries [1], central european countries [1] 

Czech Republic, Poland, Russia  (unique/total answers 2/3)
european countries [2], eastern european countries [1] 

Czech Republic, Germany, Slovakia  (unique/total answers 3/3)
regions receiving oil [1], european countries [1], count

George W. Bush, Wesley K. Clark  (unique/total answers 2/2)
powerful leaders [1], government figures [1] 

George W. Bush, Robert M. Gates  (unique/total answers 2/2)
statesmen... [1], american statesmen [1] 

George W. Bush, John Kerry, Philip Corsello  (unique/total answers 2/2)
politicians [1], men interested politics [1] 

George W. Bush, Hank Wallace, John Kerry, John Spitler  (unique/total answers 1/2)
politicians [2] 

George E. Pataki, Larry A. Silverstein, Norman Foster  (unique/total answers 2/2)
businessmen [1], american professionals [1] 

George E. Pataki, Jon S. Corzine  (unique/total answers 1/2)
politicians [2] 

Gary Weiss, Michael Lewis, Patrick Byrne  (unique/total answers 1/2)
journalists [2] 

Gail Berman, Lloyd Braun, Marc Graboff  (unique/total answers 2/2)
media executives. [1], media executives [1] 

Frank A. Dunn, MaryAnne E. Pahapill  (unique/total answers 2/2)
nortel networks employees [1], defendants [1] 

Francoise Cachin, Jean Clair, Roland Recht  (unique

Overall most frequent answers


In [21]:
overall_answers = defaultdict(int)

for _, _, answers in sorted_answers:
    flattened_answers = [answer for answer_list in answers for answer in answer_list]
    for answer in flattened_answers:
        overall_answers[answer] += 1

overall_answers = [(count, answer) for answer, count in overall_answers.items()]
overall_answers = sorted(overall_answers, reverse=True)
for count, answer in overall_answers:
    print(answer + ': ' + str(count))

politicians: 338
countries: 206
political powers: 183
nations: 174
entertainers: 147
american politicians: 131
asian countries: 116
middle eastern countries: 87
americans: 85
western asian countries: 82
actors: 72
european countries: 62
men: 61
businessmen: 59
artists: 53
states: 42
world powers: 41
writers: 39
regions: 39
companies: 39
western asia countries: 33
african countries: 32
democrats: 31
political figures: 28
american states: 24
people involved case: 22
new yorkers: 22
lawyers: 22
musicians: 21
authors: 21
journalists: 20
athletes: 20
cities: 19
performers: 17
organizations: 17
neighboring countries: 17
european areas: 17
leaders: 16
architects: 16
new york politicians: 15
american cities: 15
american actors: 15
east asian countries: 14
defendants: 14
american businessmen: 14
technology companies: 13
criminals: 13
russians: 12
directors: 12
south asian countries: 11
north american countries: 11
middle eastern regions: 11
family members: 11
basketball players: 11
american art

Answers for the most frequent tuples accross different contexts:

In [22]:
for _, entities, _ in sorted_answers[:100]:
    print(entities, ':')
    
    res_res = []
    for id_ in valid_ids:
        if entities == ', '.join(tasks[id_].entities):
            res = []
            
            for result in results[id_]:
                if not result.bug:
                    answer1 = ' '.join([w for w in result.answer1.lower().split() if w not in stopwords])
                    answer2 = ' '.join([w for w in result.answer2.lower().split() if w not in stopwords]) if result.answer2 else None

                    res.append(answer1)
                    res.append(answer2) if answer2 else None
            
            res = sorted(res)
            res_res.append(res)
            
    flatten_res_res = [r for l in res_res for r in l]
    count_res_res = sorted(set([(flatten_res_res.count(a), a) for a in flatten_res_res]), reverse=True)
    
    n = int(len(count_res_res)/2)
    to_exclude = [a for _, a in count_res_res[:n]]
    
    for res in res_res:
        res_excluded = [a if a not in to_exclude else '_' for a in res]
        print(', '.join(set([r+' ['+str(res_excluded.count(r))+']' if res_excluded.count(r)>1 else r for r in res_excluded])))
    
    print()
    

Iran, Iraq :
western asia nations, _ [3]
western asia countires, _ [3]
_ [2]
, _ [3]
_ [4]
_ [4]
_ [4]
west asian countries, _ [3]
_ [4]
_ [3]
_ [3]
_ [3]
political powers, _ [3]
_ [4]
neighboring contries, _ [3]
_ [2], middle east nations
_ [3]
_, neighboring hotspots, republic iraq
_ [3]
_ [3]
_ [4]
middle eastern republics, middle east nations..., _
_ [4]
_ [2], neighboring middle eastern countries
_ [4]

Israel, Lebanon :
mediterranean countries, _ [2], middle east countries
conflicted countries, _ [3]
_ [3]
_ [4]
neighbors syria, _ [2], countries western asia
_ [4]
_ [4]
_ [2], conflicting countries
_ [3]
_ [4]
middle eastern rivals, middle eastern nations, _ [3]
_ [3]
_ [3]
mid east countries, _ [2]
_ [4]
_ [4]
_ [3]
_ [3]
_ [3]
_ [4]

Iran, Russia :
countries asia, _ [3]
anti-west nations, _ [3]
_ [2], asian locations
_ [4]
_ [3], allied countries
_ [4]
_ [4], countries.
_ [5]
_ [4], foreign allies
political powers, _ [2], countries united states always get along

Joseph I. Lieb

In [32]:
for id_, results_list in results.items():
    for result in results_list:
        print(result.batch)

batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
batch_00
b

batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
batch_03
b

batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
batch_02
b

batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
batch_05
b